In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [16]:
df = pd.read_csv(r'D:\data\face_datafev2013.csv')

In [17]:
df.shape

(28709, 2305)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28709 entries, 0 to 28708
Columns: 2305 entries, emotion to 2303
dtypes: int64(2305)
memory usage: 504.9 MB


In [19]:
df.head()

,emotion,0,1,2,3,4,5,6,7,8,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,0,70,80,82,72,58,58,60,63,54,...,159,182,183,136,106,116,95,106,109,82
1,0,151,150,147,155,148,133,111,140,170,...,105,108,95,108,102,67,171,193,183,184
2,2,231,212,156,164,174,138,161,173,182,...,104,138,152,122,114,101,97,88,110,152
3,4,24,32,36,30,32,23,19,20,30,...,174,126,132,132,133,136,139,142,143,142
4,6,4,0,0,0,0,0,0,0,0,...,12,34,31,31,31,27,31,30,29,30


In [20]:
X_train = df.iloc[:,1:]
y_train = df['emotion']

In [21]:
X_train = X_train/255

In [22]:
X_train = X_train.values.reshape(-1,48,48,1)

In [23]:
X_train.shape

(28709, 48, 48, 1)

In [24]:
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical

In [25]:
y_train = to_categorical(y_train)
y_train.shape

(28709, 7)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.1, random_state=42)

In [28]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout, Activation
from keras.optimizers import Adam, RMSprop

In [29]:
model = Sequential()

In [30]:
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer = 'he_normal', input_shape=(48,48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer = 'he_normal', input_shape=(48,48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [31]:
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='Same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [32]:
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [33]:
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [34]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [35]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [36]:
model.add(Dense(7,kernel_initializer = 'he_normal'))
model.add(Activation('softmax'))

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 24, 24, 32)       

In [38]:
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 

In [43]:
checkpoint = ModelCheckpoint('Emotion_test.h5',
                             monitor = 'val_loss',
                             mode = 'min',
                             save_best_only = True,
                             verbose = 1)

In [44]:
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 9,
                          verbose = 1,
                          restore_best_weights = True)

In [45]:
reduce_lr =  ReduceLROnPlateau(monitor = 'val_loss',
                               factor = 0.2,
                               patience = 3,
                               verbose = 1,
                               min_delta = 0.0001)

In [46]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [47]:
model.compile(optimizer =Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
epochs=25
batch_size = 32

In [52]:
from keras.preprocessing.image import ImageDataGenerator

In [53]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=30,
        shear_range = 0.3, # randomly rotate images in the range 5 degrees
        zoom_range = 0.3, # Randomly zoom image 5%
        width_shift_range=0.4,  # randomly shift images horizontally 5%
        height_shift_range=0.4,  # randomly shift images vertically 5%
        horizontal_flip=True,  # randomly flip images
        fill_mode = 'nearest',
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              callbacks = callbacks,
                              validation_data = (X_val,y_val),
                              steps_per_epoch=X_train.shape[0] // batch_size)


Epoch 1/25
807/807 [==============================] - 1705s 2s/step - loss: 2.2155 - accuracy: 0.1944 - val_loss: 1.7755 - val_accuracy: 0.2557

Epoch 00001: val_loss improved from inf to 1.77548, saving model to Emotion_test.h5
Epoch 2/25
807/807 [==============================] - 1517s 2s/step - loss: 1.8359 - accuracy: 0.2311 - val_loss: 1.7607 - val_accuracy: 0.2692

Epoch 00002: val_loss improved from 1.77548 to 1.76073, saving model to Emotion_test.h5
Epoch 3/25
807/807 [==============================] - 1583s 2s/step - loss: 1.7955 - accuracy: 0.2529 - val_loss: 1.7362 - val_accuracy: 0.2766

Epoch 00003: val_loss improved from 1.76073 to 1.73618, saving model to Emotion_test.h5
Epoch 4/25
807/807 [==============================] - 1386s 2s/step - loss: 1.7860 - accuracy: 0.2561 - val_loss: 1.7177 - val_accuracy: 0.2950

Epoch 00004: val_loss improved from 1.73618 to 1.71772, saving model to Emotion_test.h5
Epoch 5/25
807/807 [==============================] - 1534s 2s/step - l